In [53]:
import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')
%matplotlib inline

import os

Читаем файл, преобразуем все в числовые признаки

In [54]:
df = pd.read_csv('Econom_Cities_data.csv', sep=';')

for col in ['Work', 'Price', 'Salary']:
    df[col] = df[col].astype(str).str.replace(',', '.').astype(float)

Убираем выбросы и стандартизируем

In [55]:
from sklearn.preprocessing import StandardScaler

df_no_outliers = df[(df['City'] != 'Cairo') & (df['City'] != 'Jakarta')].copy()

scaler = StandardScaler()

X_no_outliers = df_no_outliers[['Work', 'Price', 'Salary']].copy()
X_no_outliers[['Work', 'Price', 'Salary']] = scaler.fit_transform(X_no_outliers[['Work', 'Price', 'Salary']])

Запустим DBSCAN с стандартными параметрами

In [56]:
from sklearn.cluster import DBSCAN


dbscan_1 = DBSCAN()

dbscan_1.fit(X_no_outliers)

dbscan_1.labels_

array([ 0, -1, -1, -1,  0, -1, -1, -1, -1,  0,  0, -1, -1, -1, -1, -1, -1,
       -1, -1, -1,  0, -1, -1, -1, -1, -1,  0,  0, -1, -1, -1, -1, -1,  0,
       -1, -1, -1, -1, -1,  0, -1, -1, -1, -1,  0, -1], dtype=int64)

In [57]:
pd.Series(dbscan_1.labels_).value_counts()

-1    36
 0    10
Name: count, dtype: int64

Данные кластеризовались неправильно - в первый кластер попало 10 значений, а остальные считаются выбросами

In [58]:
eps_1 = [0.95, 1, 1.25, 1.5, 1.75, 2, 2.25, 2.5, 2.75, 3, 3.25, 3.5, 3.75, 4]
min_samples_1 = [2, 3, 4, 5, 6, 7, 8, 9, 10]

In [59]:
from sklearn import metrics

sil_avg = []
max_value = float("-inf")
best_params = None

for i in range(len(eps_1)):
    for j in range(len(min_samples_1)):

        db = DBSCAN(min_samples=min_samples_1[j], eps=eps_1[i], metric='euclidean').fit(X_no_outliers)
        labels = db.labels_

        n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)

        if n_clusters_ > 1:
            silhouette_avg = metrics.silhouette_score(X_no_outliers, labels)
            sil_avg.append(silhouette_avg)
            
            if silhouette_avg > max_value:
                max_value = silhouette_avg
                best_params = (eps_1[i], min_samples_1[j], n_clusters_, silhouette_avg)

print("epsilon=", best_params[0],
      "\nmin_samples=", best_params[1],
      "\nnumber of clusters=", best_params[2],
      "\naverage silhouette score= %.4f" % best_params[3])

epsilon= 0.95 
min_samples= 5 
number of clusters= 2 
average silhouette score= 0.3778


Если использовать метрику silouette и искать оптимальные параметры, то получаем 2 кластера. Посмотрим на них

In [60]:
db_scan_best_silhouette = DBSCAN(eps=best_params[0], min_samples=best_params[1]).fit(X_no_outliers)
pd.Series(db_scan_best_silhouette.labels_).value_counts()

 0    19
 1    19
-1     8
Name: count, dtype: int64

In [61]:
labels = pd.Series(db_scan_best_silhouette.labels_, name='cluster', index=X_no_outliers.index)
clusters = pd.concat([X_no_outliers.copy(), labels], axis=1)
clusters[['Work', 'Price', 'Salary']] = scaler.inverse_transform(clusters[['Work', 'Price', 'Salary']])
clusters.groupby('cluster').mean()

,Work,Price,Salary
cluster,,,
-1,1900.375000,99.925000,61.262500
0,1792.000000,77.526316,55.157895
1,1959.210526,50.115789,14.789474


0 кластер - люди работают менеьше, получают и тратят больше

1 кластер - люди работают больше, получают и тратят меньше

In [62]:
from make_map import plot_clusters_on_map

plot_clusters_on_map(df_no_outliers, clusters, 'dbscan_clusters_map_2.html')

Методом проб и ошибок были найдены гиперпараметры, которые позвоялют получить 4 класса. Посмотрим на них

In [76]:
db_scan_improved = DBSCAN(eps=0.8, min_samples=2).fit(X_no_outliers)
print(pd.Series(db_scan_improved.labels_).value_counts())

 0    19
 1    19
-1     4
 2     2
 3     2
Name: count, dtype: int64


In [80]:
labels2 = pd.Series(db_scan_improved.labels_, name='cluster', index=X_no_outliers.index)
clusters2 = pd.concat([X_no_outliers.copy(), labels2], axis=1)
clusters2[['Work', 'Price', 'Salary']] = scaler.inverse_transform(clusters2[['Work', 'Price', 'Salary']])
clusters2.groupby('cluster').mean()

,Work,Price,Salary
cluster,,,
-1,2051.250000,93.600000,42.375000
0,1792.000000,77.526316,55.157895
1,1959.210526,50.115789,14.789474
2,1874.000000,97.950000,95.150000
3,1625.000000,114.550000,65.150000


В 0 кластер попали города, в которых люди живут в средних условиях с точки зрения количества рабочих часов, зарплат и стоимости жизни.

Во 1 кластер попали города, в которых люди много работают, но мало получают. Стоимость жизни тоже маленькая.

Во второй кластер попали города, в которых цены и зарплаты сбаланисированы.

В третий кластер попали самые дорогие города, причем уровеень зарплат и количетсво рабочих часов в них достаточно невелики

In [81]:
plot_clusters_on_map(df_no_outliers, clusters2, 'dbscan_clusters_map_4.html')